In [1]:
from neo4j.v1 import GraphDatabase, basic_auth
driver = GraphDatabase.driver("bolt://localhost:7687", auth=basic_auth("neo4j", "neo4J"))
session = driver.session()

In [2]:
#Standard NS: 0
#File NS: 6
#Media NS: -2

#we're going to ignore everything else 
inPage = False
def processPage(title, ft, ID):
    if ft == 0:
        session.run("CREATE (a:article {title: {title}, id:{id}})", {"title": title, "id": ID})
    elif ft == 6: 
        session.run("CREATE (a:file {title: {title}, id:{id}})", {"title": title, "id": ID})
    elif ft == -2:
        session.run("CREATE (a:media {title: {title}, id:{id}})", {"title": title, "id": ID})
    
with open('enwiki-20170420-pages-articles.xml', 'r') as wiki:
    c = 1
    pc = 0
    title = ""
    ft = None
    ID = None
    isRedirect = None
    for line in wiki:
        line = line.strip()
        if c %  100000 == 0 and not inPage:
            session.close()
            session = driver.session()
        if line == '</page>':
            inPage = False
            c += 1
            if not isRedirect:
                processPage(title, ft, ID)
        elif inPage:
            if pc == 1:
                #Grab Title 
                title = line[7:-8].lower()
                pc += 1
            elif pc == 2:
                #Figure out namespace 
                ft = int(line[4:-5])
                pc += 1
            elif pc == 3:
                ID = int(line[4:-5])
                pc += 1
            elif pc == 4:
                if line[0:9] == "<redirect":
                    isRedirect = True
        elif line == '<page>':
            inPage = True
            isRedirect=False
            pc = 1

In [77]:
session.close() 
t

" [[Taoism|Taoist]] philosopher [[Laozi]],&lt;ref name=&quot;EB1910&quot;&gt;Peter Kropotkin, [http://dwardmac.pitzer.edu/Anarchist_Archives/kropotkin/britanniaanarchy.html &quot;Anarchism&quot;], ''[[Encyclopædia Britannica Eleventh Edition|Encyclopædia Britannica]]'' 1910.&lt;/ref&gt; and in later centuries by [[Zhuang Zhou|Zhuangzi]] and Bao Jingyan.&lt;ref name=&quot;wordpress&quot;&gt;{{cite web|url=http://robertgraham.wordpress.com/anarchism-a-documentary-history-of-libertarian-ideas-volume-one-from-anarchy-to-anarchism-300ce-1939/|title=Anarchism: A Documentary History of Libertarian I"

In [3]:
#Now, lets go through the file again and create links between the articles. 
#One thing that would have been smart would have been to do this above while I was going through every line
#And as I was giong through every line, I could have saved a YAML or XML file with title -> [links] 
#So it would have made processing several times faster
#Unfortunately, we hve to exclude sidebars. 
import re 
#PATTERN \[\[File\:(\w|\d|\s|\-|\'|_)*\.\w*\|.|\[\[(\w|\d|\s|\-|\||'|\(|\))*\]\]
inPage = False
pattern = re.compile("\[\[File\:(\w|\d|\s|\-|\'|_)*\.\w*\|.|\[\[(\w|\d|\s|\-|\||'|\(|\))*\]\]")
def regex(ft, ID, text):
    session = driver.session()
    search = "MATCH (a:"
    if ft == 0:
        search = ''.join([search, 'article {id: {id}}), '])
    elif ft == 6: 
        search = ''.join([search, 'file {id: {id}}), '])
    result = re.finditer(pattern, text)
    for k in result: 
        links = k.group(0)[2:-2].split('|')
        for l in links: 
            l = l.lower()
            if l[0:5].lower() == "file:": 
                session.run(''.join([search, '(b:file {title: {title}}) CREATE \
                (a)-[r:file_link {source: a.id, target: b.id}]->(b)']), {"id": ID, "title": l})
            else:
                session.run(''.join([search, '(b:article {title: {title}}) CREATE \
                (a)-[r:article_link {source: a.id, target: b.id}]->(b)']), {"id": ID, "title": l})
    session.close()


In [5]:
session = driver.session()
inPage = False
with open('enwiki-20170420-pages-articles.xml', 'r') as wiki:
    c = 1
    pc = 0
    ft = 0
    ID = 0
    isRedirect = None
    for line in wiki:
        line = line.strip()
        if c %  100000 == 0 and not inPage:
            print(c)
        if line == '</page>':
            inPage = False
            c += 1
        elif inPage:
            if pc == 1:
                pc += 1
            elif pc == 2:
                #Figure out namespace 
                ft = int(line[4:-5])
                pc += 1
            elif pc == 3:
                ID = int(line[4:-5])
                pc += 1
            elif pc == 4:
                if line[0:9] == "<redirect":
                    isRedirect = True
                pc += 1
            elif pc > 4:
                if not isRedirect and ft == 0 or ft == 6:
                    regex(ft, ID, line)
        elif line == '<page>':
            inPage = True
            isRedirect=False
            pc = 1

KeyboardInterrupt: 